Mackenzie Butler
Oct 11 2024
API Data Report

In thisreport I want to compare the popularity between an albums where one is full of good songs, like a 10/10 no skip album, and an album that has a one hit wonder and see how that effects the rest of the popularity in the album. Compare overall popularity of a solid album and one that contains a one hit wonder. I chose Pink Floyd's Dark Side of the Moon and Fountains of Wayne's Welcome Interstate Managers which contains the song Stacy's Mom. I think that the popularity of the albumn with the one hit wonder would drag up the average popularity score and make it possibly higher than Dark Side of the Moon


The SPotify API endpoints I need to use are tracks,album id, audio features, track features. Tracks give me access to the albumn id. I can then go in from the albumn id and get the audio features. The audio features gives me the ids to access the different track features. This is where it gives me the artist's name, albumn name, and the popularity score for each song on the albumn which i can then use to get the average popularity to compare each albumn with. Artitsts name and alumn name are constant and reliable. The popularity score was done by spotify and it's unkown the parameters they used in order to conclude this score.s?

In [53]:
import urllib
import json
import pandas as pd
import requests
import base64

In [54]:
def get_session_token(SessionID, SessionKey):
    url = 'https://accounts.spotify.com/api/token'
    data = {'grant_type':'client_credentials'}
    encoded_key = base64.b64encode(str(SessionID+':'+SessionKey).encode("ascii"))
    header = {'Authorization': 'Basic {}'.format(encoded_key.decode("ascii"))}
    response = requests.post(url, data = data, headers=header)
    print(response.status_code)
    return response.json()['access_token']

In [55]:
keys = pd.read_csv(r"C:\Users\mackb\Data in EMAT\Spotify_Keys.txt")

In [56]:
get_session_token(keys['Client_ID'].iloc[0],keys['Client_Secret'].iloc[0])

200


'BQAo-h3bwjABGKGgwPuGrvCjq0_y6_mAlvISkEypNJdzxWPOgsSM0L_AiT3OaaCS_hjp4VOd2OFUj5uEIEU0S18-G2NDOfTfoIfGp9dVM5S9euS8Dgc'

In [57]:
access_token = get_session_token(keys['Client_ID'].iloc[0],keys['Client_Secret'].iloc[0])

200


In [58]:
t_features_ep = 'https://api.spotify.com/v1/audio-features'
track_ep = 'https://api.spotify.com/v1/tracks'
ab_track_ep = 'https://api.spotify.com/v1/albums/{}/tracks'

In [59]:
def api_call(endpoint_url, api_header):
    response = requests.get(endpoint_url, headers = api_header)
    print(response.status_code)
    return response.json()

In [60]:
api_call

<function __main__.api_call(endpoint_url, api_header)>

In [61]:
session_header = {'Authorization': 'Bearer {}'.format(access_token)}

In [62]:
pf_album_id = '4LH4d3cOWNNsVw41Gqt2kv'
fw_album_id = '6TZp52tXShLQbq8yNMxqNT'

How good an album is is subjective, but Dark Side of The Moon has many hits on the album and I personally think it is a good album. I chose Stacy's Mom because I personally think Fountain's of Wayne have other great songs besides that one. I also treid to stay close to the same genre of generally rock to make the comparison.

In [63]:
ab_track_ep.format(pf_album_id)

'https://api.spotify.com/v1/albums/4LH4d3cOWNNsVw41Gqt2kv/tracks'

In [64]:
pf_album_response = api_call(ab_track_ep.format(pf_album_id), session_header)
fw_album_response = api_call(ab_track_ep.format(fw_album_id), session_header)

200
200


In [65]:
pf_album_response.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [66]:
pf_album_df = pd.DataFrame(pf_album_response['items'])
fw_album_df = pd.DataFrame(fw_album_response['items'])

In [67]:
pf_track_ids = ','.join(pf_album_df['id'].to_list())
fw_track_ids = ','.join(fw_album_df['id'].to_list())

In [68]:
pf_track_features = api_call(t_features_ep + '?ids={}'.format(pf_track_ids), session_header)
fw_track_features = api_call(t_features_ep + '?ids={}'.format(fw_track_ids), session_header)

200
200


In [69]:
pf_track_features.keys()
fw_track_features.keys()

dict_keys(['audio_features'])

In [70]:
pf_track_df = pd.DataFrame(pf_track_features['audio_features'])
pf_track_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.592,0.0195,1,-33.350,1,0.0369,0.361,0.8390,0.1080,0.0313,119.504,audio_features,574y1r7o2tRA009FW0LE7v,spotify:track:574y1r7o2tRA009FW0LE7v,https://api.spotify.com/v1/tracks/574y1r7o2tRA...,https://api.spotify.com/v1/audio-analysis/574y...,65314,4
1,0.431,0.3730,11,-15.142,0,0.0346,0.389,0.7280,0.1430,0.2530,128.153,audio_features,2ctvdKmETyOzPb2GiJJT53,spotify:track:2ctvdKmETyOzPb2GiJJT53,https://api.spotify.com/v1/tracks/2ctvdKmETyOz...,https://api.spotify.com/v1/audio-analysis/2ctv...,169535,4
2,0.389,0.5630,9,-21.191,1,0.0622,0.650,0.9030,0.1040,0.1340,165.393,audio_features,73OIUNKRi2y24Cu9cOLrzM,spotify:track:73OIUNKRi2y24Cu9cOLrzM,https://api.spotify.com/v1/tracks/73OIUNKRi2y2...,https://api.spotify.com/v1/audio-analysis/73OI...,225385,4
3,0.359,0.4800,9,-11.790,1,0.0841,0.521,0.0043,0.1780,0.3550,120.285,audio_features,3TO7bbrUKrOSPGRTB5MeCz,spotify:track:3TO7bbrUKrOSPGRTB5MeCz,https://api.spotify.com/v1/tracks/3TO7bbrUKrOS...,https://api.spotify.com/v1/audio-analysis/3TO7...,413947,4
4,0.274,0.2170,5,-13.814,1,0.0345,0.767,0.8960,0.0832,0.1810,116.334,audio_features,2TjdnqlpwOjhijHCwHCP2d,spotify:track:2TjdnqlpwOjhijHCwHCP2d,https://api.spotify.com/v1/tracks/2TjdnqlpwOjh...,https://api.spotify.com/v1/audio-analysis/2Tjd...,283872,4


In [71]:
fw_track_df = pd.DataFrame(fw_track_features['audio_features'])
fw_track_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.497,0.717,1,-5.727,1,0.0310,0.004440,0.000000,0.0994,0.306,97.962,audio_features,6sPuuzPOih8vLC92qLpdJB,spotify:track:6sPuuzPOih8vLC92qLpdJB,https://api.spotify.com/v1/tracks/6sPuuzPOih8v...,https://api.spotify.com/v1/audio-analysis/6sPu...,202320,4
1,0.574,0.879,9,-4.594,1,0.0680,0.072100,0.000000,0.0528,0.837,131.983,audio_features,6ZSf7RLIbwYsz9cz6KY6VA,spotify:track:6ZSf7RLIbwYsz9cz6KY6VA,https://api.spotify.com/v1/tracks/6ZSf7RLIbwYs...,https://api.spotify.com/v1/audio-analysis/6ZSf...,186493,4
2,0.774,0.750,11,-4.927,0,0.0412,0.002070,0.000018,0.0808,0.927,118.015,audio_features,27L8sESb3KR79asDUBu8nW,spotify:track:27L8sESb3KR79asDUBu8nW,https://api.spotify.com/v1/tracks/27L8sESb3KR7...,https://api.spotify.com/v1/audio-analysis/27L8...,197987,4
3,0.675,0.520,3,-9.515,1,0.0317,0.543000,0.000256,0.0965,0.701,117.008,audio_features,11PUdjgVfGEZLG9zs9UeTp,spotify:track:11PUdjgVfGEZLG9zs9UeTp,https://api.spotify.com/v1/tracks/11PUdjgVfGEZ...,https://api.spotify.com/v1/audio-analysis/11PU...,180093,4
4,0.438,0.910,0,-5.023,1,0.0383,0.000127,0.000008,0.3050,0.543,97.985,audio_features,0JJDHpxmlGfVS00SUAkvSF,spotify:track:0JJDHpxmlGfVS00SUAkvSF,https://api.spotify.com/v1/tracks/0JJDHpxmlGfV...,https://api.spotify.com/v1/audio-analysis/0JJD...,246347,4


In [72]:
pf_track_info = api_call(track_ep + '?market=US&ids={}'.format(pf_track_ids), session_header)
fw_track_info = api_call(track_ep + '?market=US&ids={}'.format(fw_track_ids), session_header)

200
200


These are tables that tell us the different features that spotify scored about each on track on their respective album

In [123]:
pf_features_df = pd.DataFrame(pf_track_features['audio_features'])
fw_features_df = pd.DataFrame(fw_track_features['audio_features'])

In [124]:
pf_tracks_df = pd.DataFrame(pf_track_info['tracks'])
fw_tracks_df = pd.DataFrame(fw_track_info['tracks'])
pf_tracks_df.head()

,album,artists,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,is_playable,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,65314,True,{'isrc': 'GBN9Y1100076'},{'spotify': 'https://open.spotify.com/track/57...,https://api.spotify.com/v1/tracks/574y1r7o2tRA...,574y1r7o2tRA009FW0LE7v,False,True,Speak to Me,57,https://p.scdn.co/mp3-preview/e90a9d168993a59b...,1,track,spotify:track:574y1r7o2tRA009FW0LE7v
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,169534,False,{'isrc': 'GBN9Y1100077'},{'spotify': 'https://open.spotify.com/track/2c...,https://api.spotify.com/v1/tracks/2ctvdKmETyOz...,2ctvdKmETyOzPb2GiJJT53,False,True,Breathe (In the Air),71,https://p.scdn.co/mp3-preview/85e2ac92a4028e14...,2,track,spotify:track:2ctvdKmETyOzPb2GiJJT53
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,225384,False,{'isrc': 'GBN9Y1100078'},{'spotify': 'https://open.spotify.com/track/73...,https://api.spotify.com/v1/tracks/73OIUNKRi2y2...,73OIUNKRi2y24Cu9cOLrzM,False,True,On the Run,58,https://p.scdn.co/mp3-preview/420a1bfaa06c4699...,3,track,spotify:track:73OIUNKRi2y24Cu9cOLrzM
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,413947,False,{'isrc': 'GBN9Y1100079'},{'spotify': 'https://open.spotify.com/track/3T...,https://api.spotify.com/v1/tracks/3TO7bbrUKrOS...,3TO7bbrUKrOSPGRTB5MeCz,False,True,Time,68,https://p.scdn.co/mp3-preview/af750f68023549d4...,4,track,spotify:track:3TO7bbrUKrOSPGRTB5MeCz
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,283872,False,{'isrc': 'GBN9Y1100080'},{'spotify': 'https://open.spotify.com/track/2T...,https://api.spotify.com/v1/tracks/2TjdnqlpwOjh...,2TjdnqlpwOjhijHCwHCP2d,False,True,The Great Gig in the Sky,70,https://p.scdn.co/mp3-preview/a12c96145fa9ce6b...,5,track,spotify:track:2TjdnqlpwOjhijHCwHCP2d


In [75]:
fw_tracks_df.head()

,album,artists,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,is_playable,name,popularity,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,202320,False,{'isrc': 'USESC0300001'},{'spotify': 'https://open.spotify.com/track/6s...,https://api.spotify.com/v1/tracks/6sPuuzPOih8v...,6sPuuzPOih8vLC92qLpdJB,False,True,Mexican Wine,38,1,track,spotify:track:6sPuuzPOih8vLC92qLpdJB
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,186493,True,{'isrc': 'USESC0300002'},{'spotify': 'https://open.spotify.com/track/6Z...,https://api.spotify.com/v1/tracks/6ZSf7RLIbwYs...,6ZSf7RLIbwYsz9cz6KY6VA,False,True,Bright Future In Sales,36,2,track,spotify:track:6ZSf7RLIbwYsz9cz6KY6VA
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,197986,False,{'isrc': 'USESC0300016'},{'spotify': 'https://open.spotify.com/track/27...,https://api.spotify.com/v1/tracks/27L8sESb3KR7...,27L8sESb3KR79asDUBu8nW,False,True,Stacy's Mom,72,3,track,spotify:track:27L8sESb3KR79asDUBu8nW
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,180093,False,{'isrc': 'USESC0300003'},{'spotify': 'https://open.spotify.com/track/11...,https://api.spotify.com/v1/tracks/11PUdjgVfGEZ...,11PUdjgVfGEZLG9zs9UeTp,False,True,Hackensack,42,4,track,spotify:track:11PUdjgVfGEZLG9zs9UeTp
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,246346,False,{'isrc': 'USESC0300004'},{'spotify': 'https://open.spotify.com/track/0J...,https://api.spotify.com/v1/tracks/0JJDHpxmlGfV...,0JJDHpxmlGfVS00SUAkvSF,False,True,No Better Place,27,5,track,spotify:track:0JJDHpxmlGfVS00SUAkvSF


This is the table that conatins the popularity score for the tracks on the album

In [76]:

pf_tracks_df['artists']= "Pink Floyd"
fw_tracks_df['artists']= "Fountains of Wayne"
pf_tracks_df['album']= "Dark Side of the Moon"
fw_tracks_df['album']= "Welcome Interstate Managers"

In [78]:
pf_tidy = pf_tracks_df.drop(labels=['disc_number','duration_ms','explicit','external_ids','external_urls','href','id','preview_url','is_local','is_playable','type','uri'], axis=1)
pf_tidy

,album,artists,name,popularity,track_number
0,Dark Side of the Moon,Pink Floyd,Speak to Me,57,1
1,Dark Side of the Moon,Pink Floyd,Breathe (In the Air),71,2
2,Dark Side of the Moon,Pink Floyd,On the Run,58,3
3,Dark Side of the Moon,Pink Floyd,Time,68,4
4,Dark Side of the Moon,Pink Floyd,The Great Gig in the Sky,70,5
5,Dark Side of the Moon,Pink Floyd,Money,69,6
6,Dark Side of the Moon,Pink Floyd,Us and Them,64,7
7,Dark Side of the Moon,Pink Floyd,Any Colour You Like,58,8
8,Dark Side of the Moon,Pink Floyd,Brain Damage,64,9
9,Dark Side of the Moon,Pink Floyd,Eclipse,59,10


In [80]:
fw_tidy = fw_tracks_df.drop(labels=['disc_number','duration_ms','explicit','external_ids','external_urls','href','id','is_local','is_playable','type','uri'], axis=1)
fw_tidy

,album,artists,name,popularity,track_number
0,Welcome Interstate Managers,Fountains of Wayne,Mexican Wine,38,1
1,Welcome Interstate Managers,Fountains of Wayne,Bright Future In Sales,36,2
2,Welcome Interstate Managers,Fountains of Wayne,Stacy's Mom,72,3
3,Welcome Interstate Managers,Fountains of Wayne,Hackensack,42,4
4,Welcome Interstate Managers,Fountains of Wayne,No Better Place,27,5
5,Welcome Interstate Managers,Fountains of Wayne,Valley Winter Song,31,6
6,Welcome Interstate Managers,Fountains of Wayne,All Kinds Of Time,37,7
7,Welcome Interstate Managers,Fountains of Wayne,Little Red Light,25,8
8,Welcome Interstate Managers,Fountains of Wayne,Hey Julie,48,9
9,Welcome Interstate Managers,Fountains of Wayne,Halley's Waitress,23,10


I dropped the columns that weren't relevant to the question.

In [81]:
artist_compare = pd.concat([pf_tidy, fw_tidy], axis = 0)
artist_compare

,album,artists,name,popularity,track_number
0,Dark Side of the Moon,Pink Floyd,Speak to Me,57,1
1,Dark Side of the Moon,Pink Floyd,Breathe (In the Air),71,2
2,Dark Side of the Moon,Pink Floyd,On the Run,58,3
3,Dark Side of the Moon,Pink Floyd,Time,68,4
4,Dark Side of the Moon,Pink Floyd,The Great Gig in the Sky,70,5
5,Dark Side of the Moon,Pink Floyd,Money,69,6
6,Dark Side of the Moon,Pink Floyd,Us and Them,64,7
7,Dark Side of the Moon,Pink Floyd,Any Colour You Like,58,8
8,Dark Side of the Moon,Pink Floyd,Brain Damage,64,9
9,Dark Side of the Moon,Pink Floyd,Eclipse,59,10


In [84]:
pf_average=pf_tracks_df['popularity'].mean()
pf_average

63.8

In [85]:
fw_average=fw_tracks_df['popularity'].mean()
fw_average

31.11764705882353

I wanted the average popularity to be shown to see how the popularity of the album overall was effected.

In [88]:
pf_album_popularity = pd.DataFrame({pf_tidy['artists'].iloc[0], pf_tidy['album'].iloc[0],pf_average})
pf_album_popularity

,0
0,Pink Floyd
1,63.8
2,Dark Side of the Moon


In [87]:
fw_album_popularity = pd.DataFrame({fw_tidy['artists'].iloc[0],fw_tidy['album'].iloc[0], fw_average})
fw_album_popularity

,0
0,Fountains of Wayne
1,Welcome Interstate Managers
2,31.117647


In [89]:
fw_album_popularity.loc["1", "0", "2"]
fw_album_popularity

IndexingError: Too many indexers

I am trying to switch the places of artist and album for Fountans of Wayne data frame so the same catagories are being compared in the same column. I have no ideo why it is different to being with and I can't figure out how to fix it. Everytime I back it seems a different thing is out of order and I don't know why or how to fix it

In [90]:
album_compare = pd.concat([pf_album_popularity, fw_album_popularity], axis = 1)
album_compare

,0,0
0,Pink Floyd,Fountains of Wayne
1,63.8,Welcome Interstate Managers
2,Dark Side of the Moon,31.117647


I will probably reorganize this and lable it to make it more readable

In [91]:
album_final_compare = album_compare.transpose()
album_final_compare

,0,1,2
0,Pink Floyd,63.8,Dark Side of the Moon
0,Fountains of Wayne,Welcome Interstate Managers,31.117647


In [92]:
columns_titles = ["Artist","Album","Popularity Score"]
df=album_final_compare.reindex(columns=columns_titles)
df

,Artist,Album,Popularity Score
0,NaN,NaN,NaN
0,NaN,NaN,NaN


In [93]:
album_final_compare.rename(columns={'0': 'Artist', '1': 'Album', '2' : 'Popularity Score'}, inplace=True)
album_final_compare

,0,1,2
0,Pink Floyd,63.8,Dark Side of the Moon
0,Fountains of Wayne,Welcome Interstate Managers,31.117647


I can't figure out how to rename the columns but I tried. I think it's because I flipped the columns that the numbers at the top are the index numbers so they probably can't be changes, but this is the final data frame,keep in mind I still couldn't switch the artist and album column for Fountains of Wayne so they should be mentally switched, I'm sorry. Different things keep switching

I want to compare Pink Floyd to a more popular one hit wonder so I chose Take on Me bu A-ha because it had around 2 billion streams

In [94]:
aha_album_id = '1ER3B6zev5JEAaqhnyyfbf'

In [95]:
ab_track_ep.format(aha_album_id)

'https://api.spotify.com/v1/albums/1ER3B6zev5JEAaqhnyyfbf/tracks'

In [96]:
aha_album_response = api_call(ab_track_ep.format(aha_album_id), session_header)

200


In [97]:
aha_album_response.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [98]:
aha_album_df = pd.DataFrame(aha_album_response['items'])

In [99]:
aha_track_ids = ','.join(aha_album_df['id'].to_list())

In [100]:
aha_track_features = api_call(t_features_ep + '?ids={}'.format(aha_track_ids), session_header)

200


In [101]:
aha_track_features.keys()

dict_keys(['audio_features'])

In [102]:
aha_track_df = pd.DataFrame(aha_track_features['audio_features'])
aha_track_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.573,0.902,6,-7.638,0,0.0540,0.0180,0.001250,0.0928,0.876,84.412,audio_features,2WfaOiMkCvy7F5fcp2zZ8L,spotify:track:2WfaOiMkCvy7F5fcp2zZ8L,https://api.spotify.com/v1/tracks/2WfaOiMkCvy7...,https://api.spotify.com/v1/audio-analysis/2Wfa...,225280,4
1,0.669,0.616,9,-15.981,1,0.0402,0.0361,0.013200,0.0629,0.824,139.523,audio_features,5CXuHc7GfGJOCMvMn3G4r7,spotify:track:5CXuHc7GfGJOCMvMn3G4r7,https://api.spotify.com/v1/tracks/5CXuHc7GfGJO...,https://api.spotify.com/v1/audio-analysis/5CXu...,254533,4
2,0.513,0.676,9,-6.148,0,0.0309,0.3010,0.000000,0.0864,0.311,116.592,audio_features,5tewIdMVsaJWN19ZnmnPNN,spotify:track:5tewIdMVsaJWN19ZnmnPNN,https://api.spotify.com/v1/tracks/5tewIdMVsaJW...,https://api.spotify.com/v1/audio-analysis/5tew...,228693,4
3,0.286,0.665,5,-15.820,1,0.0475,0.0178,0.004050,0.1530,0.695,113.721,audio_features,22Ca1a5rQ5g2UFEZ3pp4tL,spotify:track:22Ca1a5rQ5g2UFEZ3pp4tL,https://api.spotify.com/v1/tracks/22Ca1a5rQ5g2...,https://api.spotify.com/v1/audio-analysis/22Ca...,156733,4
4,0.582,0.368,6,-16.596,0,0.0297,0.0122,0.000305,0.0776,0.491,111.809,audio_features,13WPj6r2DVNU3GoZtMc9eU,spotify:track:13WPj6r2DVNU3GoZtMc9eU,https://api.spotify.com/v1/tracks/13WPj6r2DVNU...,https://api.spotify.com/v1/audio-analysis/13WP...,302867,4


In [103]:
aha_track_info = api_call(track_ep + '?market=US&ids={}'.format(aha_track_ids), session_header)

200


In [104]:
aha_features_df = pd.DataFrame(aha_track_features['audio_features'])

In [105]:
aha_tracks_df = pd.DataFrame(aha_track_info['tracks'])
aha_tracks_df.head()

,album,artists,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,is_playable,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,225280,False,{'isrc': 'USWB19901214'},{'spotify': 'https://open.spotify.com/track/2W...,https://api.spotify.com/v1/tracks/2WfaOiMkCvy7...,2WfaOiMkCvy7F5fcp2zZ8L,False,True,Take on Me,86,https://p.scdn.co/mp3-preview/d1427dd0a300eecc...,1,track,spotify:track:2WfaOiMkCvy7F5fcp2zZ8L
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,254533,False,{'isrc': 'USWB19901215'},{'spotify': 'https://open.spotify.com/track/5C...,https://api.spotify.com/v1/tracks/5CXuHc7GfGJO...,5CXuHc7GfGJOCMvMn3G4r7,False,True,Train of Thought,42,https://p.scdn.co/mp3-preview/f06c62b06029dcf1...,2,track,spotify:track:5CXuHc7GfGJOCMvMn3G4r7
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,228693,False,{'isrc': 'USWB19901216'},{'spotify': 'https://open.spotify.com/track/5t...,https://api.spotify.com/v1/tracks/5tewIdMVsaJW...,5tewIdMVsaJWN19ZnmnPNN,False,True,Hunting High and Low,61,https://p.scdn.co/mp3-preview/c4446806f6376911...,3,track,spotify:track:5tewIdMVsaJWN19ZnmnPNN
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,156733,False,{'isrc': 'USWB19901217'},{'spotify': 'https://open.spotify.com/track/22...,https://api.spotify.com/v1/tracks/22Ca1a5rQ5g2...,22Ca1a5rQ5g2UFEZ3pp4tL,False,True,The Blue Sky,38,https://p.scdn.co/mp3-preview/ef9743279aabccea...,4,track,spotify:track:22Ca1a5rQ5g2UFEZ3pp4tL
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,302866,False,{'isrc': 'USWB19901218'},{'spotify': 'https://open.spotify.com/track/13...,https://api.spotify.com/v1/tracks/13WPj6r2DVNU...,13WPj6r2DVNU3GoZtMc9eU,False,True,Living a Boy's Adventure Tale,45,https://p.scdn.co/mp3-preview/11ad54296f34d543...,5,track,spotify:track:13WPj6r2DVNU3GoZtMc9eU


In [112]:
aha_tracks_df['artists']= "A-ha"
aha_tracks_df['album']= "Hunting High and Low"

In [118]:
aha_tidy = aha_tracks_df.drop(labels=['disc_number','duration_ms','explicit','artist','external_ids','external_urls','href','id','preview_url','is_local','is_playable','type','uri'], axis=1)
aha_tidy

,album,artists,name,popularity,track_number
0,Hunting High and Low,A-ha,Take on Me,86,1
1,Hunting High and Low,A-ha,Train of Thought,42,2
2,Hunting High and Low,A-ha,Hunting High and Low,61,3
3,Hunting High and Low,A-ha,The Blue Sky,38,4
4,Hunting High and Low,A-ha,Living a Boy's Adventure Tale,45,5
5,Hunting High and Low,A-ha,The Sun Always Shines on T.V.,63,6
6,Hunting High and Low,A-ha,And You Tell Me,32,7
7,Hunting High and Low,A-ha,Love Is Reason,38,8
8,Hunting High and Low,A-ha,I Dream Myself Alive,37,9
9,Hunting High and Low,A-ha,Here I Stand and Face the Rain,34,10


In [119]:
artist_compare2 = pd.concat([pf_tidy, aha_tidy], axis = 0)
artist_compare2

,album,artists,name,popularity,track_number
0,Dark Side of the Moon,Pink Floyd,Speak to Me,57,1
1,Dark Side of the Moon,Pink Floyd,Breathe (In the Air),71,2
2,Dark Side of the Moon,Pink Floyd,On the Run,58,3
3,Dark Side of the Moon,Pink Floyd,Time,68,4
4,Dark Side of the Moon,Pink Floyd,The Great Gig in the Sky,70,5
5,Dark Side of the Moon,Pink Floyd,Money,69,6
6,Dark Side of the Moon,Pink Floyd,Us and Them,64,7
7,Dark Side of the Moon,Pink Floyd,Any Colour You Like,58,8
8,Dark Side of the Moon,Pink Floyd,Brain Damage,64,9
9,Dark Side of the Moon,Pink Floyd,Eclipse,59,10


In [120]:
aha_average=aha_tracks_df['popularity'].mean()
aha_average

47.6

In [121]:
aha_album_popularity = pd.DataFrame({aha_tidy['artists'].iloc[0], aha_tidy['album'].iloc[0],aha_average})
aha_album_popularity

,0
0,A-ha
1,Hunting High and Low
2,47.6


In [122]:
album_compare2 = pd.concat([pf_album_popularity, aha_album_popularity], axis = 1)
album_compare2

,0,0
0,Pink Floyd,A-ha
1,63.8,Hunting High and Low
2,Dark Side of the Moon,47.6


again some things are being switched around so they aren't in the same column

For both data sets we got the same results, A good and popular album has a higher popularity score than the one with one hit wonders, even considereing Aha's had around 2 billion streams so it is pretty cool.


It is a little hard because we don't know what perameters they used to make the popularity score. Another way to maybe see the difference would be to add up all the plays/streams in each resepective albumn and find the average and compare to which one has been listened to more.